In [1]:
import os
import torch
from tqdm import tqdm_notebook
from matplotlib import pyplot as plt
from itertools import product
import json
from collections import defaultdict
import random

from torch import nn
from torch.autograd import Variable
from torch.functional import F
from torchvision import models
import torchvision
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np
from torchviz import make_dot


%matplotlib inline
%reload_ext autoreload
%autoreload 2

# from fastai import transforms, model, dataset, conv_learner

from PIL import ImageDraw, ImageFont
from matplotlib import patches, patheffects
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

torch.set_printoptions(precision=3)

from SSD_model import get_SSD_model
from VOC_data import VOC_dataset
from draw_img_utils import *
from SSDloss import *

torch.set_printoptions(precision=3)

PATH = '/home/kindeqi/PyTorch_SSD/dataset/VOCdevkit/VOC2007'
trn_anno_path = '/home/kindeqi/PyTorch_SSD/annotation/PASCAL_VOC/pascal_train2007.json'
val_anno_path = '/home/kindeqi/PyTorch_SSD/annotation/PASCAL_VOC/pascal_val2007.json'

train_dataset = VOC_dataset(PATH, trn_anno_path)
batch_size = 16
learning_rate = 5e-4
num_iter = 10
vgg_weight_path = '/home/kindeqi/.torch/models/vgg16-397923af.pth'

In [22]:
def detection_collate_fn(batch):
    imgs, bboxes, labels = [], [], []
    for i, b, l in batch:
        imgs.append(i); bboxes.append(b); labels.append(l)
    return torch.stack(imgs), bboxes, labels

trn_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=detection_collate_fn)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

model = get_SSD_model(batch_size, vgg_weight_path)
model = model.to(device)

# optimizer = torch.optim.SGD(params = model.parameters(), lr=1e-4, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for _ in range(num_iter):
    
    prior_box = get_prior_box()
    
    for i, batch in enumerate(trn_dataloader):
        
#         just the first 10 batch
        if i >= 20:
            continue
        
        imgs, bboxes, labels = batch
        imgs = imgs.to(device)
        cls_preds, loc_preds = model(imgs)

        model.zero_grad()

        total_loss = 0
        total_loc_loss, total_cls_loss = 0, 0

        for idx in range(imgs.shape[0]):

            img, bbox, label = imgs[idx], bboxes[idx], labels[idx]
            cls_pred, loc_pred = cls_preds[idx], loc_preds[idx]

            # img, bbox, label = tmp
            # img, bbox, label = train_dataset[0]

            # img = img.unsqueeze(0)

            # cls_pred, loc_pred = model(img)
            # cls_pred, loc_pred = cls_pred.squeeze(0), loc_pred.squeeze(0)

            # PATH = 'C:\\datasets\\pascal\\'
            # anno_path = f'{PATH}PASCAL_VOC\\pascal_train2007.json'
            # train_dataset = VOC_dataset(PATH, anno_path)

            # img, bbox, label = train_dataset[7]
            # img = img.unsqueeze(0)

#             prior_box = get_prior_box()
            iou = get_iou(bbox, prior_box)

            pos_mask, cls_target, bbox_target = get_target(iou, prior_box, img, bbox, label)
            pos_mask, cls_target, bbox_target = pos_mask.to(device), cls_target.to(device), bbox_target.to(device)

            # model = get_SSD_model(1)
            # cls_pred, loc_pred = model(img)
            # cls_pred, loc_pred = cls_pred.squeeze(0), loc_pred.squeeze(0)

            loss_loc, loss_cls = loss(cls_pred, loc_pred, pos_mask, cls_target, bbox_target)
            total_loc_loss += loss_loc; total_cls_loss += loss_cls

#             total_loss += (loss_loc + loss_cls)
            total_loss += loss_cls

        total_loss /= float(batch_size)
        total_loss.backward()

        optimizer.step()
        if i % 10 == 0:
            cls_loss = round(float(total_cls_loss / batch_size), 3)
            loc_loss = round(float(total_loc_loss / batch_size), 3)
            total_loss = round(float(total_loss), 3)
            
            print(i, 'cls_loss: {}, loc_loss: {}, loss: {}'.format(cls_loss, loc_loss, total_loss))
        
        if i == 20:
            torch.save(model.state_dict(), '20.pth')
        
        if i == 50:
            torch.save(model.state_dict(), '50.pth')
            
        if i == 100:
            torch.save(model.state_dict(), '100.pth')


0 cls_loss: 3.025, loc_loss: 4.629, loss: 3.025
10 cls_loss: 3.034, loc_loss: 4.447, loss: 3.034
0 cls_loss: 2.75, loc_loss: 4.618, loss: 2.75
10 cls_loss: 2.729, loc_loss: 4.431, loss: 2.729
0 cls_loss: 2.539, loc_loss: 4.631, loss: 2.539
10 cls_loss: 2.674, loc_loss: 4.43, loss: 2.674
0 cls_loss: 2.409, loc_loss: 4.641, loss: 2.409
10 cls_loss: 2.595, loc_loss: 4.43, loss: 2.595
0 cls_loss: 2.404, loc_loss: 4.631, loss: 2.404
10 cls_loss: 2.729, loc_loss: 4.429, loss: 2.729
0 cls_loss: 2.475, loc_loss: 4.62, loss: 2.475
10 cls_loss: 2.516, loc_loss: 4.433, loss: 2.516
0 cls_loss: 2.418, loc_loss: 4.626, loss: 2.418
10 cls_loss: 2.619, loc_loss: 4.432, loss: 2.619
0 cls_loss: 2.363, loc_loss: 4.627, loss: 2.363
10 cls_loss: 2.567, loc_loss: 4.432, loss: 2.567
0 cls_loss: 2.372, loc_loss: 4.622, loss: 2.372
10 cls_loss: 2.613, loc_loss: 4.432, loss: 2.613
0 cls_loss: 2.457, loc_loss: 4.62, loss: 2.457
10 cls_loss: 2.557, loc_loss: 4.431, loss: 2.557


In [38]:
img, bbox, label = train_dataset[11]
img = img.unsqueeze(0).to(device)

conf_pred, loc_pred = model(img)

In [39]:
prior_box = get_prior_box()
iou = get_iou(bbox, prior_box)

pos_mask, cls_target, bbox_target = get_target(iou, prior_box, img, bbox, label)

In [40]:
cls = F.softmax(conf_pred[0][pos_mask])

/home/kindeqi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


In [41]:
cls.max(dim=1)

(tensor([0.133, 0.143, 0.134, 0.145, 0.196, 0.125, 0.141, 0.221, 0.168, 0.189,
         0.354, 0.206, 0.124, 0.109, 0.133, 0.123], device='cuda:0',
        grad_fn=<MaxBackward0>),
 tensor([ 7,  7,  7,  7,  7,  7,  7,  7, 19, 15, 13, 19, 19, 19, 12, 19],
        device='cuda:0'))

In [42]:
cls_target[pos_mask]

tensor([9, 9, 9, 9, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7], dtype=torch.int32)

In [39]:
model.base_net[12].weight.shape

torch.Size([256, 256, 3, 3])

In [40]:
model_20 = get_SSD_model(1, vgg_weight_path)
model_20.load_state_dict(torch.load('100.pth'))

In [41]:
img, bbox, label = train_dataset[0]

In [42]:
img = img.unsqueeze(0)

In [43]:
cls_pred, loc_pred = model_20(img)

In [44]:
prior_box = get_prior_box()
iou = get_iou(bbox, prior_box)

pos_mask, cls_target, bbox_target = get_target(iou, prior_box, img, bbox, label)

In [45]:
pred = cls_pred[0][pos_mask]

In [46]:
cls_target[pos_mask]

tensor([7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
        7, 7, 7, 7, 7, 7], dtype=torch.int32)

In [47]:
F.softmax(pred)

/home/kindeqi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


tensor([[7.446e-01, 3.591e-04, 1.342e-03, 1.275e-02, 2.664e-03, 1.323e-03,
         1.368e-04, 1.786e-03, 2.293e-03, 6.282e-03, 1.915e-03, 1.188e-04,
         2.109e-02, 6.637e-03, 1.521e-03, 1.833e-01, 2.602e-03, 6.436e-04,
         2.065e-03, 1.601e-03, 4.908e-03],
        [7.446e-01, 3.591e-04, 1.342e-03, 1.275e-02, 2.664e-03, 1.323e-03,
         1.368e-04, 1.786e-03, 2.293e-03, 6.282e-03, 1.915e-03, 1.188e-04,
         2.109e-02, 6.637e-03, 1.521e-03, 1.833e-01, 2.602e-03, 6.436e-04,
         2.065e-03, 1.601e-03, 4.908e-03],
        [7.446e-01, 3.591e-04, 1.342e-03, 1.275e-02, 2.664e-03, 1.323e-03,
         1.368e-04, 1.786e-03, 2.293e-03, 6.282e-03, 1.915e-03, 1.188e-04,
         2.109e-02, 6.637e-03, 1.521e-03, 1.833e-01, 2.602e-03, 6.436e-04,
         2.065e-03, 1.601e-03, 4.908e-03],
        [7.446e-01, 3.591e-04, 1.342e-03, 1.275e-02, 2.664e-03, 1.323e-03,
         1.368e-04, 1.786e-03, 2.293e-03, 6.282e-03, 1.915e-03, 1.188e-04,
         2.109e-02, 6.637e-03, 1.521e-03, 1.83

In [48]:
bbox_target[pos_mask]

tensor([[ 4.767,  6.817,  1.074, -0.955],
        [ 4.767,  7.649,  1.074, -0.955],
        [ 4.767,  8.065,  1.074, -0.955],
        [ 5.599,  3.072,  1.074, -0.955],
        [ 5.599,  3.488,  1.074, -0.955],
        [ 5.599,  4.320,  1.074, -0.955],
        [ 6.431, -1.089,  1.074, -0.955],
        [ 6.431, -0.257,  1.074, -0.955],
        [ 6.431,  0.159,  1.074, -0.955],
        [ 7.263, -4.834,  1.074, -0.955],
        [ 7.263, -4.418,  1.074, -0.955],
        [ 7.263, -3.585,  1.074, -0.955],
        [ 7.263,  6.817,  1.074, -0.955],
        [ 7.263,  7.649,  1.074, -0.955],
        [ 7.263,  8.065,  1.074, -0.955],
        [ 9.260,  1.992, -0.656,  0.781],
        [11.186,  3.829,  0.289,  1.726],
        [ 9.260,  3.169, -0.656,  0.781],
        [ 9.623,  1.647, -0.464, -2.493],
        [11.186, 13.075,  0.289,  1.726],
        [ 9.260, 10.823, -0.656,  0.781],
        [11.186, 14.497,  0.289,  1.726],
        [ 9.260, 12.000, -0.656,  0.781],
        [ 9.623,  6.235, -0.464, -

In [49]:
loc_pred[0][pos_mask]

tensor([[ 5.631e+00,  7.717e-01,  2.746e-01,  2.131e-03],
        [ 5.631e+00,  7.717e-01,  2.746e-01,  2.131e-03],
        [ 5.631e+00,  7.717e-01,  2.746e-01,  2.131e-03],
        [ 5.631e+00,  7.717e-01,  2.746e-01,  2.131e-03],
        [ 5.631e+00,  7.717e-01,  2.746e-01,  2.131e-03],
        [ 5.631e+00,  7.717e-01,  2.746e-01,  2.131e-03],
        [ 5.631e+00,  7.717e-01,  2.746e-01,  2.131e-03],
        [ 5.631e+00,  7.717e-01,  2.746e-01,  2.131e-03],
        [ 5.631e+00,  7.717e-01,  2.746e-01,  2.131e-03],
        [ 5.631e+00,  7.717e-01,  2.746e-01,  2.131e-03],
        [ 5.631e+00,  7.717e-01,  2.746e-01,  2.131e-03],
        [ 5.631e+00,  7.717e-01,  2.746e-01,  2.131e-03],
        [ 5.631e+00,  7.717e-01,  2.746e-01,  2.131e-03],
        [ 5.631e+00,  7.717e-01,  2.746e-01,  2.131e-03],
        [ 5.631e+00,  7.717e-01,  2.746e-01,  2.131e-03],
        [ 9.183e+00,  1.652e+00,  7.651e-01,  9.103e-01],
        [ 1.106e+01,  1.067e+00, -4.055e-01,  6.250e-01],
        [ 9.18

In [14]:
s = slice(None, 20, None)

In [16]:
for i in range(s.start, s.stop):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
